# **Efficient Yelp API Calls**

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

### **Credentials and Accessing the API**

In [2]:
# Load API Credentials
with open('/Users/NelLouw/.secret/yelp_api.json') as f:   
    login = json.load(f)
    
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

### **Define Search**

Use the Yelp API to search your favorite city for a cuisine type of your choice. <br>
Extract all of the results from your search and compile them into one dataframe using a for loop

In [3]:
# Set our API call parameters 
LOCATION = 'NY,NY'
TERM = 'Noodles' # Search Term #

### **Create a JSON file where your results will be saved**

In [4]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_NY_Noodles.json"
JSON_FILE

'Data/results_in_progress_NY_Noodles.json'

- #### **Create a results-in-progress JSON file, but only if it doesn't exist**

In [5]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_NY_Noodles.json already exists.


- #### **Determine how many results are already in the file**

In [6]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


- #### **Figure out how many pages of results we will need**

In [7]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [8]:
## How many results total?
total_results = results['total']
total_results

12000

In [9]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [10]:
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

600

- #### **Add this page of results to .json file**

In [13]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

### **Create a Function**

In [15]:
def create_json_file(JSON_FILE, delete_if_exists=False):
    # Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)

    # If it DOES exist:
    if file_exists == True:
        # Check if user wants to delete if exists
        if delete_if_exists:
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            # delete file
            os.remove(JSON_FILE)
            # Recursive call to function after old file deleted
            create_json_file(JSON_FILE, delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")
    
    # If it does NOT exist:
    else:
        # INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")

        # CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)

        # If JSON_FILE included a folder:
        if len(folder) > 0:
            # create the folder
            os.makedirs(folder, exist_ok=True)
        
        # Save empty list to start the json file
        with open(JSON_FILE, 'w') as f:
            json.dump([], f)

# Example usage:
# create_json_file("example.json", delete_if_exists=True)

In [16]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[!] Data/results_in_progress_NY_Noodles.json already exists. Deleting previous file...
[i] Data/results_in_progress_NY_Noodles.json not found. Saving empty list to new file.
- 0 previous results found.


595

In [17]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

# Exceeded 1000 api calls. Stopping loop.

  0%|          | 0/595 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [18]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,oe8GEFE4QLFAKt87y7zcgA,very-fresh-noodles-new-york,Very Fresh Noodles,https://s3-media2.fl.yelpcdn.com/bphoto/gRqfWs...,False,https://www.yelp.com/biz/very-fresh-noodles-ne...,1483,"[{'alias': 'taiwanese', 'title': 'Taiwanese'},...",4.5,"{'latitude': 40.74207, 'longitude': -74.00565}","[delivery, pickup]",$$,"{'address1': '409 W 15th St', 'address2': None...",+13322156161,(332) 215-6161,4183.626316
1,lynQoI3w_pzYfHGeuUU-Qg,shu-jiao-fu-zhou-new-york-2,Shu Jiao Fu Zhou,https://s3-media2.fl.yelpcdn.com/bphoto/Nmkgn1...,False,https://www.yelp.com/biz/shu-jiao-fu-zhou-new-...,1333,"[{'alias': 'chinese', 'title': 'Chinese'}]",4.5,"{'latitude': 40.7174109, 'longitude': -73.9921...","[delivery, pickup]",$,"{'address1': '295 Grand St', 'address2': None,...",+12126252532,(212) 625-2532,1343.336119
2,utVNm8OfDl_H154lMlcMlA,zhongzhong-noodles-brooklyn-brooklyn-3,Zhongzhong Noodles - Brooklyn,https://s3-media3.fl.yelpcdn.com/bphoto/u8Dy1H...,False,https://www.yelp.com/biz/zhongzhong-noodles-br...,9,"[{'alias': 'chinese', 'title': 'Chinese'}, {'a...",5.0,"{'latitude': 40.69301, 'longitude': -73.97198}","[delivery, pickup]",NaN,"{'address1': '154 Adelphi St', 'address2': Non...",,,2344.671670
3,97q5nUQTuBo_h00XCxna2A,tasty-hand-pulled-noodles-new-york,Tasty Hand-Pulled Noodles,https://s3-media4.fl.yelpcdn.com/bphoto/FJttNY...,False,https://www.yelp.com/biz/tasty-hand-pulled-noo...,1864,"[{'alias': 'chinese', 'title': 'Chinese'}, {'a...",4.0,"{'latitude': 40.7142, 'longitude': -73.99787}","[delivery, pickup]",$$,"{'address1': '1 Doyers St', 'address2': '', 'a...",+12127911817,(212) 791-1817,1017.728241
4,7bpZZUpEclhn29wWVqFsKA,lucky-rabbit-noodles-brooklyn,Lucky Rabbit Noodles,https://s3-media2.fl.yelpcdn.com/bphoto/hD5Y2U...,False,https://www.yelp.com/biz/lucky-rabbit-noodles-...,53,"[{'alias': 'tapasmallplates', 'title': 'Tapas/...",4.5,"{'latitude': 40.70375, 'longitude': -73.98809}","[delivery, pickup]",$$,"{'address1': '140 Plymouth St', 'address2': ''...",+13479160098,(347) 916-0098,550.701426


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,B9Mbi12NpBJkZcRtQNtMBA,malaysia-grill-new-york,Malaysia Grill,https://s3-media2.fl.yelpcdn.com/bphoto/1VH4B8...,False,https://www.yelp.com/biz/malaysia-grill-new-yo...,351,"[{'alias': 'malaysian', 'title': 'Malaysian'}]",4.0,"{'latitude': 40.79948, 'longitude': -73.96775}","[pickup, delivery]",$$,"{'address1': '224 W 104th St', 'address2': '',...",+12125791333,(212) 579-1333,10683.588057
996,IqgiEIUVzVrHuhNTy-mcJw,soup-kiosk-new-york,Soup Kiosk,https://s3-media3.fl.yelpcdn.com/bphoto/a1lCsM...,False,https://www.yelp.com/biz/soup-kiosk-new-york?a...,70,"[{'alias': 'foodstands', 'title': 'Food Stands...",4.0,"{'latitude': 40.72457, 'longitude': -73.99878}",[delivery],$,"{'address1': '94 Prince St', 'address2': None,...",,,2158.990822
997,hZDZ--W5gd7E0W8lz_feLg,abc-eats-food-hall-long-island-city,ABC Eats Food Hall,https://s3-media1.fl.yelpcdn.com/bphoto/kLmdwb...,False,https://www.yelp.com/biz/abc-eats-food-hall-lo...,42,"[{'alias': 'bubbletea', 'title': 'Bubble Tea'}...",4.0,"{'latitude': 40.745231028373816, 'longitude': ...","[pickup, delivery]",$$,"{'address1': '22-22 Jackson Ave', 'address2': ...",+17183925737,(718) 392-5737,5967.994581
998,AILf_L7I9vJ2pyHmSo9v8Q,obao-new-york-4,OBAO,https://s3-media4.fl.yelpcdn.com/bphoto/Xopn2B...,False,https://www.yelp.com/biz/obao-new-york-4?adjus...,1186,"[{'alias': 'asianfusion', 'title': 'Asian Fusi...",4.0,"{'latitude': 40.75717754, 'longitude': -73.968...","[pickup, delivery]",$$,"{'address1': '222 E 53rd St', 'address2': '', ...",+12123085588,(212) 308-5588,6147.662373
999,mnXrPqOVbX_06D9OAo4dLA,atti-brooklyn,Atti,https://s3-media4.fl.yelpcdn.com/bphoto/SVPvsI...,False,https://www.yelp.com/biz/atti-brooklyn?adjust_...,73,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.5,"{'latitude': 40.68841151989921, 'longitude': -...","[pickup, delivery]",$$$$,"{'address1': '294 Livingston St', 'address2': ...",+19179091122,(917) 909-1122,2123.675412
